In [ ]:
 pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data = pd.read_excel('Review Canva.xlsx')
data.head()

,Unnamed: 0,content,score
0,0,keren mantap,5
1,1,mudah untuk di pelajari,4
2,2,Aplikasi nya keren dan banyak juga logo logo n...,5
3,3,sangat membantu sekali untuk para guru dan par...,5
4,4,bagus sekali,4


In [ ]:
# Inisialisasi objek preprocessing
stop_words = set(stopwords.words('indonesian'))

# Membaca slangword dari file .txt
def load_slangwords(file_path):
    with open(file_path, 'r') as file:
        slang_dict = json.load(file)
    return slang_dict

def create_reverse_slang_dict(slang_dict):
    reverse_slang_dict = {}
    for formal, slangs in slang_dict.items():
        for slang in slangs:
            reverse_slang_dict[slang] = formal
    return reverse_slang_dict

# Memuat slangwords
slang_dict = load_slangwords('slang_words.txt')
reverse_slang_dict = create_reverse_slang_dict(slang_dict)

def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def normalize_slangwords(text, slang_dict):
    words = text.split()
    normalized_words = [slang_dict.get(word, word) for word in words]
    return ' '.join(normalized_words)

# Membuat stemmer bahasa Indonesia
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def preprocess_text(text):
    # Ubah menjadi huruf kecil
    text = text.lower()
    # Hapus emoji
    text = remove_emojis(text)
    # Hapus angka
    text = re.sub(r'\d+', '', text)
    # Hapus tanda baca dan karakter non-alfabet
    text = re.sub(r'[^a-z\s]', '', text)
    # Normalisasi slangword
    text = normalize_slangwords(text, reverse_slang_dict)
    # Tokenisasi
    tokens = word_tokenize(text)
    # Hapus stopword
    tokens = [word for word in tokens if word not in stop_words]
    # Stemming dengan Sastrawi
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

# Terapkan preprocessing pada kolom komentar
data['clean'] = data['content'].apply(preprocess_text)

# Menampilkan data yang sudah di-preprocess
data.head()


,Unnamed: 0,content,score,clean
0,0,keren mantap,5,keren mantap
1,1,mudah untuk di pelajari,4,mudah ajar
2,2,Aplikasi nya keren dan banyak juga logo logo n...,5,aplikasi keren logo logo pilih
3,3,sangat membantu sekali untuk para guru dan par...,5,bantu guru siswa giat ajar
4,4,bagus sekali,4,bagus


In [ ]:
#Labelling

data['label'] = np.where(data['score'] >= 4, '1', '0')

In [ ]:
data.head()

,Unnamed: 0,content,score,clean,sentiment_label,label
0,0,keren mantap,5,keren mantap,1,1
1,1,mudah untuk di pelajari,4,mudah ajar,1,1
2,2,Aplikasi nya keren dan banyak juga logo logo n...,5,aplikasi keren logo logo pilih,1,1
3,3,sangat membantu sekali untuk para guru dan par...,5,bantu guru siswa giat ajar,1,1
4,4,bagus sekali,4,bagus,1,1


In [ ]:
data.to_excel('dataclean.xlsx')

In [ ]:
data.to_excel('dataclean.xlsx')

**TF-IDF Transformation**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['clean'])

In [ ]:
df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

In [ ]:
df.to_excel('tf-idf.xlsx')

**NAIVE BAYES CLASSIFIER**

In [ ]:
from pandas import DataFrame
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold, ShuffleSplit, StratifiedShuffleSplit
from sklearn.naive_bayes import BernoulliNB
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
Y = pd.DataFrame(data['sentiment_label'])

In [ ]:
# Naive Bayes Classifier
Y_new = DataFrame.to_numpy(Y)
Y_new = Y_new.astype(int)
X_new = DataFrame.to_numpy(df)
kfold= StratifiedKFold(n_splits=10, shuffle=False)
kfold.get_n_splits(X_new)
kfold.get_n_splits(Y_new)

clf = BernoulliNB()
akurasi = []

for train, test in kfold.split(X_new, Y_new):
  (X_new[train], X_new[test])
  (Y_new[train].ravel(), Y_new[test].ravel())
  hitung = clf.fit(X_new[train], Y_new[train].ravel())
  predictnbc1 = clf.predict(X_new[train])
  predictnbc = clf.predict(X_new[test])
  fpr1, tpr1, _1 = metrics.roc_curve(Y_new[train].ravel(), predictnbc1)
  fpr, tpr, _ = metrics.roc_curve(Y_new[test].ravel(), predictnbc)

  print("\nConfusiion Matrix Train: \n", metrics.confusion_matrix(Y_new[train], predictnbc1))
  print("\nConfusiion Matrix Test: \n", metrics.confusion_matrix(Y_new[test], predictnbc))
  print()
  print(metrics.classification_report(Y_new[train], predictnbc1))
  print(metrics.classification_report(Y_new[test], predictnbc))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 8 members, which is less than n_splits=10.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:1020: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu


Confusiion Matrix Train: 
 [[ 1  7]
 [ 0 81]]

Confusiion Matrix Test: 
 [[10]]

              precision    recall  f1-score   support

           0       1.00      0.12      0.22         8
           1       0.92      1.00      0.96        81

    accuracy                           0.92        89
   macro avg       0.96      0.56      0.59        89
weighted avg       0.93      0.92      0.89        89

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        10

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10


Confusiion Matrix Train: 
 [[ 1  6]
 [ 0 82]]

Confusiion Matrix Test: 
 [[0 1]
 [0 9]]

              precision    recall  f1-score   support

           0       1.00      0.14      0.25         7
           1       0.93      1.00      0.96        82

    accuracy                           0.93        89
   macro a

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif